In [1]:
# If needed (only once per environment):
# %pip install pandas sqlalchemy python-dotenv matplotlib seaborn

import os
from pathlib import Path
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

import matplotlib.pyplot as plt
import seaborn as sns

# Notebook display tweaks
plt.rcParams["figure.figsize"] = (9, 5)
sns.set_theme()

# Load env (expects DB_URL in your .env at project root)
load_dotenv("../.env")  # adjust path if your .env is at project root

DB_URL = os.getenv("DB_URL")
assert DB_URL, "DB_URL not set. Make sure .env is configured."

engine = create_engine(DB_URL)

# Output folders
BASE_REPORT_DIR = Path("../reports") 
PLOTS_DIR = BASE_REPORT_DIR / "plots"
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

RUN_TS = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H-%M-%SZ")

def savefig(name: str):
    out = PLOTS_DIR / f"{RUN_TS}_{name}.png"
    plt.tight_layout()
    plt.savefig(out, dpi=150, bbox_inches="tight")
    print(f"[saved plot] {out}")
    plt.close()



In [2]:
with engine.connect() as conn:
    patients       = pd.read_sql(text('SELECT * FROM silver."patients"'), conn)
    prescriptions  = pd.read_sql(text('SELECT * FROM silver."prescriptions"'), conn)
    billing        = pd.read_sql(text('SELECT * FROM silver."billing"'), conn)
    appointments   = pd.read_sql(text('SELECT * FROM silver."appointments"'), conn)
    doctors        = pd.read_sql(text('SELECT * FROM silver."doctors"'), conn)


In [5]:
if "doctor_id" in appointments.columns:
    appt_counts = (
        appointments.groupby("doctor_id", dropna=False)
        .size()
        .reset_index(name="appointments_per_doctor")
    )   

    plt.figure(figsize=(6, 7))
    sns.boxplot(y=appt_counts["appointments_per_doctor"])
    plt.title("Appointments per Doctor (Boxplot)")
    plt.ylabel("Appointments per Doctor")
    savefig("boxplot_appointments_per_doctor")
else:
    print("[warn] 'doctor_id' not in silver.appointments")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_boxplot_appointments_per_doctor.png


In [7]:
amount_col = next((c for c in ["amount","billing_amount","total_amount"] if c in billing.columns), None)

if amount_col:
    amounts = pd.to_numeric(billing[amount_col], errors="coerce").dropna()

    Q1, Q3 = amounts.quantile(0.25), amounts.quantile(0.75)
    IQR = Q3 - Q1
    upper = Q3 + 1.5 * IQR

    plt.figure()
    plt.hist(amounts, bins=40)
    plt.axvline(upper, linestyle="--")
    plt.title(f"Billing Amounts (hist) + Upper Outlier Threshold\nQ1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}, Upper={upper:.2f}")
    plt.xlabel("Amount")
    plt.ylabel("Count")
    savefig("hist_billing_amounts_with_outlier")

    outlier_count = int((amounts > upper).sum())
    summary = pd.DataFrame({
        "Q1":[Q1], "Q3":[Q3], "IQR":[IQR], "Upper_Threshold":[upper],
        "Total_Obs":[len(amounts)], "Outlier_Count":[outlier_count]
    })

else:
    print("[warn] amount column not found (looked for amount/billing_amount/total_amount)")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_hist_billing_amounts_with_outlier.png


In [8]:
# pick date column name
date_col = next((c for c in ["appointment_date","date","scheduled_date"] if c in appointments.columns), None)

if date_col:
    appt = appointments[[date_col]].copy()
    appt[date_col] = pd.to_datetime(appt[date_col], errors="coerce")
    appt = appt.dropna(subset=[date_col])

    end_date = appt[date_col].max().normalize()
    start_date = end_date - pd.offsets.MonthBegin(6)  # ~ last 6 months
    mask = (appt[date_col] >= start_date) & (appt[date_col] <= end_date)
    appt6 = appt.loc[mask].copy()

    daily = appt6.groupby(appt6[date_col].dt.date).size().reset_index(name="appointments")


    plt.figure()
    plt.plot(daily[date_col], daily["appointments"])
    plt.title(f"Daily Appointments (Last 6 Months)\n{start_date.date()} → {end_date.date()}")
    plt.xlabel("Date")
    plt.ylabel("Appointments")
    plt.xticks(rotation=45)
    savefig("line_daily_appointments_last_6m")
else:
    print("[warn] no appointment date column found (tried appointment_date/date/scheduled_date)")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_line_daily_appointments_last_6m.png


In [10]:
# Appointment Status Distribution (reuse appointments df)
plt.figure(figsize=(6,4))
appointments["status"].value_counts().plot(kind="bar")

plt.title("Appointment Status Distribution")
plt.ylabel("Count")

# save instead of show
savefig("bar_appointment_status_distribution")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_bar_appointment_status_distribution.png


In [12]:
# Medicines per Prescription (reuse prescriptions df)
presc_counts = prescriptions.groupby("prescription_id")["medicine"].count().reset_index(name="med_count")

plt.figure(figsize=(7,5))
presc_counts["med_count"].hist(bins=10)
plt.title("Medicines per Prescription")
plt.xlabel("Number of Medicines")
plt.ylabel("Count of Prescriptions")

# save instead of show
savefig("hist_medicines_per_prescription")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_hist_medicines_per_prescription.png


In [13]:
# Billing Amount Outliers (reuse billing df)
plt.figure(figsize=(5,7))
billing["amount"].dropna().plot(kind="box")

plt.title("Billing Amount Outliers")
plt.ylabel("Amount")

# save instead of show
savefig("boxplot_billing_amount_outliers")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_boxplot_billing_amount_outliers.png


In [15]:
if "gender" in patients.columns:
    plt.figure(figsize=(6,4))
    patients["gender"].value_counts().plot(kind="bar", color=["skyblue","lightcoral","lightgreen"])
    plt.title("Patient Gender Distribution")
    plt.xlabel("Gender")
    plt.ylabel("Count")
    savefig("bar_patient_gender_distribution")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_bar_patient_gender_distribution.png


In [16]:
if "appointment_date" in appointments.columns:
    appt = appointments.copy()
    appt["appointment_date"] = pd.to_datetime(appt["appointment_date"], errors="coerce")
    appt = appt.dropna(subset=["appointment_date"])
    appt["weekday"] = appt["appointment_date"].dt.day_name()

    plt.figure(figsize=(8,5))
    appt["weekday"].value_counts().reindex(
        ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    ).plot(kind="bar")
    plt.title("Appointments by Weekday")
    plt.xlabel("Day of Week")
    plt.ylabel("Appointments")
    savefig("bar_appointments_by_weekday")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_bar_appointments_by_weekday.png


In [17]:
if "payment_method" in billing.columns:
    plt.figure(figsize=(7,5))
    billing.groupby("payment_method")["amount"].sum().sort_values().plot(kind="barh")
    plt.title("Total Billing Amount by Payment Method")
    plt.xlabel("Total Amount")
    plt.ylabel("Payment Method")
    savefig("barh_billing_by_payment_method")


[saved plot] ../reports/plots/2025-08-27T09-02-43Z_barh_billing_by_payment_method.png
